# Verifier Agent

In [ ]:
import requests
import json
import qrcode
import uuid
import base64
import re
import jwt
import time
# https://pypi.org/project/termcolor/
from termcolor import colored,cprint

In [ ]:
# Set the URL for the API endpoint
# Need to specify the host as host.docker.internal as the Prism Agents and the Prism Playground are running in two isolated docker environments
base_url = "http://host.docker.internal:9000"

headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

## Check Agent Records
1. Connections
2. Credentials
3. Proof Presentations

In [ ]:
print(colored('Agent State:\n', attrs=['bold']))
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

print(colored('Connection Records:\n', attrs=['bold']))
# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    
path = '/prism-agent/issue-credentials/records'

print(colored('Credential Records:\n', attrs=['bold']))
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)
    
path = '/prism-agent/present-proof/presentations'

print(colored('Proof Presentation Records:\n', attrs=['bold']))
# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

## Make a Connection with Holder Agent

In [ ]:
path = '/prism-agent/connections'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

### Create a connection record containing the invitation
```bash
curl -X 'POST' \
	'http://localhost:8080/prism-agent/connections' \
	-H 'Content-Type: application/json' \
	-d '{
		"label": "Connect with Alice"
		}' | jq
```

In [ ]:
path = '/prism-agent/connections'
data = {"label": "Connect with Alice"}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    STATE = loaded_json['state']
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")

## Send invitation to Holder

In [ ]:
invitation = json.loads(response.content.decode())
invitationUrl = invitation['invitation']['invitationUrl']
invitationUrlbase64verifier = re.sub("^.*_oob=", "", invitationUrl)
print(invitationUrlbase64verifier)
connection_id = invitation['connectionId']
%store invitationUrlbase64verifier

### Generate QR code from invitation

In [ ]:
# Link for connection invitation
input_data = invitationUrl
# Creating an instance of qrcode
qr = qrcode.QRCode(
        version=1,
        box_size=10,
        border=5)
qr.add_data(input_data)
qr.make(fit=True)
img = qr.make_image(fill='black', back_color='white')
img.save('issuer_agent_invite_QRcode.png')

from IPython.display import Image
Image(width=400, filename='./issuer_agent_invite_QRcode.png')

### Retrieving the list of connections
```bash
curl -X 'GET' 'http://localhost:8080/prism-agent/connections' | jq
```

In [ ]:
# STATE = None

path = '/prism-agent//connections/' + connection_id

print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'ConnectionResponseSent':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json['state']
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("State: {0}".format(STATE), "blue", attrs=["bold"]))

        try:
            their_did = loaded_json['theirDid']
        except (KeyError) as e:
            print(colored("Holder didn't accept invitation yet. Head over to Holder notebook and accept connection invitation".format(connection_id,STATE), "red", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


## Executing the `Verifier` flow
---
connectionId is the holder (connectionId or did)
Replace `{CONNECTION_ID}` with the DID of the holder displayed at startup in the his Prism Agent console logs

### Initiates a Proof Request

```shell
curl -X 'POST' \
  'http://localhost:8070/prism-agent/present-proof/presentations' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "connectionId": "{CONNECTION_ID}", "proofs":[]
}'
```

In [ ]:
path = '/prism-agent/present-proof/presentations'
data = {
  "connectionId": connection_id, "proofs":[]
}
# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 201:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-201 status code")
    print(response.content)

### Send proof request to Holder

In [ ]:
presentation_id = loaded_json['presentationId']
%store presentation_id
print(presentation_id)

### Check Presentation state  PresentationReceived 
> check PresentationReceived !
```shell
curl -X 'GET' 'http://localhost:8070/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```


In [ ]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    # print(response.content)
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=1))
    STATE = loaded_json[0]['status']
    # proof_requestId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")

In [ ]:
path = '/prism-agent/present-proof/presentations'

print('Current state for Presentation {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'PresentationVerified':
    # Make the request
    response = requests.get(base_url + path, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Print the response content
        loaded_json = json.loads(response.content.decode())
        # print(json.dumps(loaded_json, indent=2))
        STATE = loaded_json[0]['status']
        print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
        print(colored("State: {0}".format(STATE), "blue", attrs=["bold"]))

    else:
        # Print an error message
        print("Error: The API returned a non-200 status code")
        

    time.sleep(1)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))



## TODO - Verify JWT and display contents

### Accept PresentationReceived 
> Replace `{PRESENTATION_ID}` with the UUID of the record from the presentation records list with state PresentationReceived

```shell
curl -X 'PATCH' \
  'http://localhost:8070/prism-agent/present-proof/presentations/{PRESENTATION_ID}' \
  -H 'Content-Type: application/json' \
  -d '{"action": "presentation-accept"}' | jq
```



In [ ]:
data = {"action": "presentation-accept"}

path = '/prism-agent/present-proof/presentations/' + presentation_id

# Make the request
response = requests.patch(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    print(response.content)
    # loaded_json = json.loads(response.content.decode())
    # print(json.dumps(loaded_json, indent=2))
    # proof_requestId = loaded_json[0]['presentationId']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Check Presentation state  PresentationVerified 
> check PresentationVerified !

```shell
curl -X 'GET' 'http://localhost:8070/prism-agent/present-proof/presentations' -H 'accept: application/json' | jq
```

In [ ]:
path = '/prism-agent/present-proof/presentations'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

## DIDs

### Create a new unpublised DID stored in Prism Agent
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Content-Type: application/json' \
  --header 'Accept: application/json' \
  --data-raw '{
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }'
```

In [ ]:
path = '/prism-agent/did-registrar/dids'
data = {
    "documentTemplate": {
      "publicKeys": [
        {
          "id": "auth0",
          "purpose": "authentication"
        }
      ],
      "services": []
    }
  }

# Make the request
response = requests.post(base_url + path, headers=headers, json=data)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Lists all the DIDs in Prism Agent
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/did-registrar/dids' \
  --header 'Accept: application/json'
```



In [ ]:
path = '/prism-agent/did-registrar/dids'

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    DID_REF = loaded_json[0]['longFormDid']
    DID_REF_SHORT = loaded_json[0]['did']
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)

### Publishes the DID in Prism Agent to the blockchain
Replace `DID_REF` by the DID on Prism Agent that should be published
```bash
curl --location --request POST 'http://localhost:8080/prism-agent/did-registrar/dids/{DID_REF}/publications' \
--header 'Accept: application/json'
```

In [ ]:
path = '/prism-agent/did-registrar/dids/' + DID_REF + '/publications'

# Make the request
response = requests.post(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 202:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-202 status code")
    print(response.content)

### Resolves the DID document of Prism DID
Replace `DID_REF` by the DID on Prism Agent that should be resolved
```bash
curl --location --request GET 'http://localhost:8080/prism-agent/dids/{DID_REF}' \
--header 'Accept: application/json'
```

In [ ]:
path = '/prism-agent/dids/' + DID_REF

# Make the request
response = requests.get(base_url + path, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Print the response content
    loaded_json = json.loads(response.content.decode())
    print(json.dumps(loaded_json, indent=2))
    
else:
    # Print an error message
    print("Error: The API returned a non-200 status code")
    print(response.content)